In [21]:
%matplotlib inline
import numpy as np
from sklearn import svm
import matplotlib.pyplot as plt

In [22]:
with open("/home/christopher/data/ml/digit-recognizer/train.csv", "r") as data:
    fullTrain = np.loadtxt(data, delimiter=",", skiprows=1, dtype="float64")
with open("/home/christopher/data/ml/digit-recognizer/test.csv", "r") as data:
    fullTest = np.loadtxt(data, delimiter=",", skiprows=1, dtype="float64")

In [23]:
train = fullTrain
test = fullTest

In [24]:
trainResults = train[:,0]
trainData = train[:,1:] / 256
test = test / 256

In [25]:
clf = svm.NuSVC()#gamma=0.001, kernel="sigmoid")   
clf.fit(trainData, trainResults)

NuSVC(cache_size=200, class_weight=None, coef0=0.0,
   decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
   max_iter=-1, nu=0.5, probability=False, random_state=None,
   shrinking=True, tol=0.001, verbose=False)

In [26]:
res = clf.predict(test)

In [27]:
with open("results.csv", "w") as f:
    f.write("ImageId,Label\n")
    for i, r in enumerate(res):
        f.write("{},{}\n".format(i+1, int(r)))